# Testing likelihood runtimes


The rate limiting step for linkelihood based inference is the time taken to obtain a single likelihood sample.

We want this to be as fast as possible 

See e.g. tips on using numba: https://numba.pydata.org/numba-doc/latest/user/performance-tips.html

In [4]:
%load_ext autoreload
%autoreload 2

In [46]:
import sys
import json 
import pandas as pd 
try:
    sys.path.remove("../py_src") # Hacky way to add higher directory to python modules path. 
except:
    pass
sys.path.append("../py_src") # Means that I dont have to make src/ a proper python package

In [47]:
from system_parameters import SystemParameters
from pulsars import Pulsars
from synthetic_data import SyntheticData
from model import LinearModel
from kalman_filter import KalmanFilter
from priors import priors_dict,bilby_priors_dict
import numpy as np 
import copy 
import logging 
logger = logging.getLogger().setLevel(logging.ERROR)

#Function to create synthetic data
def create_synthetic_data(h,measurement_noise,seed):

    P    = SystemParameters(h=h,σp=None,σm=measurement_noise,seed=seed)       #define the system parameters as a class
    PTA  = Pulsars(P)               # setup the PTA
    data = SyntheticData(PTA,P)     # generate some synthetic data

    return P,PTA,data

#Fit model to data and return likelihood
def setup_kalman_likelihood(data,P,PTA,measurement_model):

    _P = copy.copy(P)
    _PTA = copy.copy(PTA)
    _P.measurement_model = measurement_model
    

    #Define the model 
    model = LinearModel(_P)

    #Initialise the Kalman filter
    KF = KalmanFilter(model,data.f_measured,_PTA)

    #Create a bilby dict 
    init_p,parameter_priors = bilby_priors_dict(_PTA,_P)
    
    sampled_parameters = parameter_priors.sample(1)
    return KF,sampled_parameters

### Create some data



In [48]:
h = 1e-12
measurement_noise = 1e-11
seed=1236
P,PTA,data = create_synthetic_data(h,measurement_noise,seed)

In [49]:
measurement_model = "earth"
KF,parameters_sample = setup_kalman_likelihood(data,P,PTA,measurement_model)


### Now time the function

In [55]:
def function_to_time(KF,params):
    KF.likelihood(params)

In [56]:
%timeit function_to_time(KF,parameters_sample)

697 ns ± 48.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [67]:
function_to_time(KF,parameters_sample)

['f00', 'f01', 'f02', 'f03', 'f04', 'f05', 'f06', 'f07', 'f08', 'f09', 'f010', 'f011', 'f012', 'f013', 'f014', 'f015', 'f016', 'f017', 'f018', 'f019', 'f020', 'f021', 'f022', 'f023', 'f024', 'f025', 'f026', 'f027', 'f028', 'f029', 'f030', 'f031', 'f032', 'f033', 'f034', 'f035', 'f036', 'f037', 'f038', 'f039', 'f040', 'f041', 'f042', 'f043', 'f044', 'f045', 'f046']
[327.84702057 205.53069911 303.09097941 326.60056194 348.55923172
 112.9497232  346.53199652 215.60881048 190.26783735 193.71568339
 238.0040532  172.64290994 125.20024517 277.93770695 317.37894193
 316.1239842  216.3733371  218.81184038 170.9373699  266.869167
 245.42611965 607.67753842 367.76712112 244.39137772 186.49407835
 465.13523835 339.31568716 200.65880529 216.17123056 130.78951613
 263.98071421 641.92822213 192.85652083 315.44356371 163.04791292
 191.45091203 345.27813646 168.09667797 420.18944325  62.29588783
 320.5922923  335.81621338 279.59658217 275.7078326  192.59196365
 290.25460815 207.96816337]


In [52]:
def blobfish():
    return 1

In [54]:
%timeit blobfish()

60.5 ns ± 0.946 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
